# Project 3: Classification - Reddit Posts

In this project, we are tasked to select 2 subreddits from [Reddit](https://www.reddit.com/), scrape the posts, use NLP and build 2 classification models to classify which post belongs to which subreddit. After the models is built, an evaluation have to be performed and we have to select the model that best answer the problem statement.

The project is split into 2 notebooks:
- First notebook gathers and cleaned the data
- Second notebook we performed exploratory data analysis, feature engineering, modelling and evaluation

# Problem Statement
**Business Problem:**<br>We have quite a number of overlapping subreddits. How can we ensure that posts in the subreddits topics are at the right place to improve user experience. 

**Data Science Problem:**<br>Create classification models to identify if a post is from the subreddit [learn python](https://www.reddit.com/r/learnpython/) (positive) or [learn programming](https://www.reddit.com/r/learnprogramming/)  (negative). Select the model you think best answers the problem and evaluate the model.   

# Executive Summary

In this project, we identified learnpython and learnprogramming subreddits, scrape data from them, built 4 classification models to identify which post belong to which subreddit, evaluate the top 2 models and select one which best answer our business needs, to identify whether a post should be in the learn python or learn programming subreddit and move the post to the respective subreddits.

We used requests to scrape the data, and identified that selftext which contains the text of the post is the best column we can use for our NLP classification problem. We then cleaned the data using BeautifulSoup, nltk's stopwords, regex and python's string manipulation. After gathering and cleaning the data, we are have a total of 1930 posts, 49.27% of them from learn programming and 50.73% from learn python. We have an almost equal ratio of post for each subreddit which is considered a good dataset to have for classification problems. From this we are also able to determine that our baseline accuracy is 50.73%

Wordcloud was created for each subreddit to visualise words that appear frequently in each subreddit and the top 20 words for the entire corpus was identified and plotted on a graph with their frequency count. 

During this phase of exploratory data analysis we discovered that there are common words which frequently appear in both subreddits. A customised stopwords list was created. This customised stopwords list contains words which our models will ignore. We then performed stemming and lemmatizing on the self text column and identified that lemma text is the best compared to the original and stemmed text.

We then built 4 models:
- Count Vectorizer with Naive Bayesian's MultinomialNB
- Tfidf Vectorizer with Naive Bayesian's MultinomialNB
- Count Vectorizer with Logistics Regression
- Tfidf Vectorizer with Logistics Regression

and found out that the Count Vectorizer with Naive Bayesian's MultinomialNB model which has a score of 78.05% in our test data answers our business problem the best as it is not overfitted and has the best precision.

Although one of the model has a higher score compared to our selected model, it do not answer our busines problem adequately. Its precision is lower compared our selected model and for our business problem, the cost of false positive is higher when compared to the cost of false negative.

For the next iteration to improve this model, we can look at running the model again after moving the posts to their respective classified subreddits to determine and check if the model performed better as we will have lesser overlapped posts due to the close nature of the selected subreddits. Or we can collecting larger dataset then tune our model again to improve its classification accuracy. We can also perform an analysis if title, author or number comments are able to make distinct classifications amongst the classes and consider adding them in as a feature to improve the model.

When the model is accurate enough, we can expand the usage to other subreddits and consider having subreddit auto-tagging function or subreddit suggestion when a thread starter is creating a post. This decreases the clutterness of information in our subreddits and allows more distinct subreddit categories for users to choose from, increasing user experience and retention, which will in turn increase traffic volume and popularity of reddit to the community.

### Contents:
- [Import Libraries](#Import-Libraries)
- [Data Gathering](#Data-Gathering)
- [Preliminary EDA and Data Cleaning](#Preliminary-EDA-and-Data-Cleaning)

# Import Libraries

In [3]:
import pandas as pd
import requests # library that helps you read url
import random
import time
import re

from bs4 import BeautifulSoup # this is for scrapping websites
from nltk.corpus import stopwords # Import the stop word list

# Data Gathering

A function was created to scrape posts from reddit.com, by passing in the url when calling the function. As reddit allows a maximum of scraping 1000 unique posts, 25 at the time, the loop is ran for 40 times.

A random agent list was created and used during each loop. A random sleep timer was also implemented to randomise the time between each loop. These preventive actions are taken so as to prevent reddit from blocking us while scraping.

In [2]:
def scrape(url):
    posts = []
    after = None
    random_agent = ['Secret Agent 9.9', 'Polly Pocket 1.1', 'Flying Dutchman 2.3', 'Thinking Soldier 3.5']
    
    for a in range(40):
        if after == None:
            current_url = url + '.json'
        else:
            current_url = url + '.json' + '?after=' + after
            
        res = requests.get(current_url, headers={'User-agent': random.choice(random_agent)})
        
        if res.status_code != 200:
            print('Status error', res.status_code)
            break
        
        current_dict = res.json()
        current_posts = [p['data'] for p in current_dict['data']['children']]
        posts.extend(current_posts)
        after = current_dict['data']['after']
        
        sleep_duration = random.randint(2,6)
        time.sleep(sleep_duration)
        
        df = pd.DataFrame(posts)
 
    return df

## Scrape Learn Python Subreddit

In [3]:
# scrape the reddit's subreddit Singapore and retrieve dataframe
learnpython_df = scrape('https://www.reddit.com/r/learnpython/')

In [4]:
learnpython_df.shape

(987, 98)

## Scrape Learn Programming Subreddit

In [5]:
learnprog_df = scrape('https://www.reddit.com/r/learnprogramming/')

In [6]:
learnprog_df.shape

(995, 99)

The number of rows we have for both subreddits are also similar, 987 for learn python and 995 for learn programming. This means that we have a good ratio of posts for our analysis. However this is still subjected to further scrutiny as we are performing NLP for classification purpose. There may be posts with only pictures in the content with no texts. 

We can also see that there is one more column in learn programming when compared to learn python. We may not be utilizing all columns in our analysis and modelling.

We will explore more on the ratio of posts and whether the additional column will affect our project.

# Save Data Frame

We will first save both dataframes as separate csv. This csv file will come in handy should we need to take a look at our raw data should we encounter any issues during other phases of the project.

In [7]:
learnpython_df.to_csv('datasets/learnpython.csv', index = False)

In [8]:
learnprog_df.to_csv('datasets/learnprog.csv', index = False)

# Preliminary EDA and Data Cleaning

We first create 2 functions which will be used for our EDA.

select_features is a simple function which allows us to select the desired columns which we think will be relevant for this project.

create_content_char_count_and_drop_zero is a function that creates a new column that holds the character count of the target column, then drop the rows which character count is zero. Parameters col_name which will be the name of the column we will create and thread_column which is the target column have been given default value which can be change should we decide to call the function for other target columns.

In [58]:
# Function to only select the desired features from the data frame

def select_features(df, features_list):
    df = df[features_list]
    return df

In [59]:
# create a column that holds the value of the character count of the thread's content and drop rows with count of 0

def create_content_char_count_and_drop_zero(df, col_name = 'content_char_count', thread_column = 'selftext'):
    df[col_name] = df[thread_column].str.len()
    df.drop(df.loc[df[col_name] == 0].index, inplace = True)

## Merge both Data Frames

We first merge both data frames as a single one for easier manipulation. 

In [60]:
combined_df = pd.concat([learnpython_df, learnprog_df], ignore_index=True, sort = False)

In [61]:
combined_df.shape

(1982, 99)

## Select Columns

In [62]:
# Return the dataframe with only the selected features from the features list

features_list = ['author', 'title', 'selftext', 'num_comments', 'permalink', 'subreddit']
combined_df = select_features(combined_df, features_list)

After exploring the reddit site and finding out what data each column holds, it is decided that the top column which will aid in our our classification problem through NLP will be selftext. We also need the subreddit column which shows which subreddit the post is from.

Title is the title of the thread, which can also be an aid to our classification problem here, however for now, we will work on selftext first. Another use of title is that we can use it as a feature to check for the uniqueness of our dataset. The possibility of having 2 posts having the exact same title will be low. Although there is another column Id which should point to the post ID which should also be unique, but since we are keeping title for now, we will use title for the uniqueness check and if this fails, we can always go back to our saved csv and retrieve the Id column.

Other columns kept were author which shows the name of the thread starter, num_comments which shows the total number of comments at the time when the scrape is performed. Similar to title, these may come in handy should we require additional features. There can be a pattern on the number of comments for each subreddit and an author can be known to be a frequent thread starter for a particular subreddit.

Permalink is the link to the post and may come in handy if we wish to see an individual post at the site.

We do not have an very large dataset, keeping these addtional 4 columns will not have a significant impact on performance. We will keep these columns for now as there may be a use for them. 

## Create Character Count Column and Drop

We will call the function to create a new column which holds the value of the character count for column selftext. After the new column is created, if the character count is 0, the row will be dropped.

In [63]:
create_content_char_count_and_drop_zero(combined_df)

## Remove Duplicates

We will check the column title for any duplicates and drop them if there is.

In [65]:
# remove duplicates in title column
combined_df.drop_duplicates(subset = 'title', inplace = True)

## Check for Null

After the above operations are performed, we will check if we have any null values in our dataframe. We can see that there are a total of 6 null values in both selftext and content_char_count, as content_char_count is calculated from selftext, we can infer that they most likely belong to the same row. 6 or the maximum 12 rows compared to our original of 1982 rows is not significant, we shall drop these rows with null values.

In [67]:
combined_df.isnull().sum()

author                0
title                 0
selftext              6
num_comments          0
permalink             0
subreddit             0
content_char_count    6
dtype: int64

In [69]:
combined_df.dropna(inplace = True)

## Encode Subreddit

We shall map the value in our subreddit column which shows which subreddit the post belongs to. This will also be our y column. learnpython will be mapped as 1 (positive) and learnprogramming will be mapped as 0 (negative).

In [70]:
combined_df['subreddit'].replace({'learnpython' : 1, 'learnprogramming' : 0}, inplace = True)

## Text Cleaning

We will create a function to clean the text of our target column. Function clean_contents takes in 2 parameters, the dataframe and the target column text. 

We will first remove any html codes like tags. Although we do not see much of these in our selftext for the rows which we manually checked, but we will still run this as we did not manually eye all rows.

Next, using regex we will remove html links, there is quite a number of links in the selftext, both subreddits are related to computer programming, we can see similar sites like stackoverflow and w3schools appearing in both subreddits. These links will not be useful features for our classification models.

We will then proceed to remove punctuations and lower the caps for all the words and split them into individual words. We will then use nltk's stopwords to remove commonly used words like 'the', 'an', 'a', 'yours' etc.

Finally we will join back the words and replace them in the respective selftext column

In [71]:
def clean_contents(df, column_text):
    # This removes html codes like <br>
    removed_html = BeautifulSoup(column_text).get_text() 
    
    # 2. This remove http followed by anything before a space, tab or newline links should not have any space or tab or newline between them
    removed_links = re.sub("http.\S+", " ", removed_html)
                                                              
    # 3. This removes non-letters
    removed_punctuation = re.sub("[^a-zA-Z]", " ", removed_links)
                                                                  
        
    # 4. Convert to lower case, split into individual words.
    lower_words = removed_punctuation.lower().split()
    
    # 4. This converts nltk's stopwords into a set
    stops = set(stopwords.words('english'))
    
    # 5. Compare our remaining words with stop words and only keep words not in the stop words
    meaningful_words = [words for words in lower_words if words not in stops]
    
    # 6. Join the words back into one string separated by space and replace the joint text back to the dataframe
    cleansed_text = " ".join(meaningful_words)
    df.replace(column_text, cleansed_text, inplace = True)
   

We will run the function for both selftext and title column. After that we checked the shape of the dataframe. We still have 1934 rows, this is still a healthy number of posts

In [72]:
for selftext in combined_df['selftext']:
    clean_contents(combined_df, selftext)

In [73]:
for title in combined_df['title']:
    clean_contents(combined_df, title)

In [74]:
combined_df.shape

(1934, 7)

After cleaning, there may be some values with 0 characters, we will run the function again on selftext and title to update the character count column for self text and create a character count column for title and drop rows with zero characters in them. A check on the shape shows that we only dropped another 4 rows and is left with 1930 rows. 

This is still a healthy number, we will then save the cleansed dataframe as another csv file and proceed to the next notebook marking the end of our data gathering and data cleaning phase.

In [75]:
create_content_char_count_and_drop_zero(combined_df)

In [76]:
create_content_char_count_and_drop_zero(combined_df, col_name = 'title_char_count', thread_column = 'title')

In [77]:
combined_df.shape

(1930, 8)

In [78]:
combined_df.to_csv('datasets/cleansed.csv', index = False)